In [1]:
## import pandas
import pandas as pd

In [2]:
## turn excel sheet into dataframe, call new dataframe
df= pd.read_excel("data-raw/nyc.xlsx")
df

,Period,Value
0,Jan-2016,17453
1,Feb-2016,16828
2,Mar-2016,16547
3,Apr-2016,17066
4,May-2016,18114
...,...,...
107,Dec-2024,25669
108,Jan-2025,21497
109,Feb-2025,23729
110,Mar-2025,24716


In [3]:
## reformat the dataframe to get rid of weird notation
pd.options.display.float_format = '{:,.0f}'.format

In [4]:
## understand data better
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Period  112 non-null    object
 1   Value   112 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.9+ KB


In [5]:
## summary of dataset and description of distribution in case I would need it
df.describe()

,Value
count,112
mean,"21,792"
std,"3,898"
min,"12,122"
25%,"18,634"
50%,"22,447"
75%,"25,106"
max,"29,776"


## CLEANING DATA

In [7]:
## one-based indexing so the data starts at 1, not 0. I used Anaconda for this because I couldn't figure out how to do it in the notes.
## I actually learned that it was discouraged to reindex because it's not reproducible and confusing, so I commented it out but wanted it to be seen in my process.
# business_applications = df.reset_index(drop=True).rename(index=lambda x: x + 1)
business_applications = df.copy()
business_applications



,Period,Value
0,Jan-2016,17453
1,Feb-2016,16828
2,Mar-2016,16547
3,Apr-2016,17066
4,May-2016,18114
...,...,...
107,Dec-2024,25669
108,Jan-2025,21497
109,Feb-2025,23729
110,Mar-2025,24716


In [8]:
# Convert the 'Period' column to datetime using pandas so it registers the column as a date, not a string. I did need a refresher on how to do this and used Anaconda.
business_applications['Period'] = pd.to_datetime(business_applications['Period'], format='%b-%Y')

In [9]:
# sort values in descending order by the 'Period' column, since I will be working with the latest data first.
business_applications.sort_values(by= "Period", ascending= False)



,Period,Value
111,2025-04-01,24699
110,2025-03-01,24716
109,2025-02-01,23729
108,2025-01-01,21497
107,2024-12-01,25669
...,...,...
4,2016-05-01,18114
3,2016-04-01,17066
2,2016-03-01,16547
1,2016-02-01,16828


In [25]:
## add year column
business_applications['Year'] = business_applications['Period'].dt.year

In [12]:
## add month column
business_applications['Month'] = business_applications['Period'].dt.month

In [13]:
## call business_applications
business_applications

,Period,Value,Month
0,2016-01-01,17453,1
1,2016-02-01,16828,2
2,2016-03-01,16547,3
3,2016-04-01,17066,4
4,2016-05-01,18114,5
...,...,...,...
107,2024-12-01,25669,12
108,2025-01-01,21497,1
109,2025-02-01,23729,2
110,2025-03-01,24716,3


In [47]:
# Pivot the data. This took a lot of trial and error. I worked with Jesus heavily on this, we couldn't figure out why the Year/Month were in the same column. But we exported the data and learned it was just a Jupityr notebook bug and it was fine.
pivoted_table = business_applications.pivot(index='Month', columns='Year', values='Value')
pivoted_table

Year,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
Month,,,,,,,,,,
1,"17,453","17,166","19,007","17,032","17,565","27,965","28,337","24,006","25,980","21,497"
2,"16,828","17,295","18,825","19,465","19,522","24,374","25,371","25,438","24,594","23,729"
3,"16,547","17,707","18,686","19,705","15,580","25,710","25,170","26,376","23,721","24,716"
4,"17,066","17,732","18,650","19,810","12,122","28,797","24,506","25,295","24,540","24,699"
5,"18,114","17,945","19,142","19,111","15,426","27,117","24,782","25,404","24,232",NaN
6,"14,718","18,588","19,167","19,526","20,748","24,706","23,803","27,108","24,437",NaN
7,"18,070","18,251","19,374","18,759","29,776","25,381","24,638","27,228","23,978",NaN
8,"16,651","18,336","18,805","18,862","28,888","25,196","24,305","26,921","23,991",NaN
9,"17,711","17,778","18,965","19,302","26,450","24,557","25,084","26,152","23,913",NaN


In [49]:
##created csv to check what the extra row is, per prior comment
pivoted_table.to_csv("example.csv")

## ANALYZING DATA

In [52]:
##I am going to create a new dataframe that holds 2019-2025 data to be safe, because ultimately, I want to be able to analyze the percentage change YOY to compare 2024 to 2025, as well as 2025 to pre-pandemic and post-pandemic times. 
## I used anaconda for this because the iloc expression we were taught in class was not working. Not fully sure I understand difference between iloc and loc.
##Data in 2025 only goes up to April.

df1 = pivoted_table.loc[:, 2019:2025]
df1

Year,2019,2020,2021,2022,2023,2024,2025
Month,,,,,,,
1,"17,032","17,565","27,965","28,337","24,006","25,980","21,497"
2,"19,465","19,522","24,374","25,371","25,438","24,594","23,729"
3,"19,705","15,580","25,710","25,170","26,376","23,721","24,716"
4,"19,810","12,122","28,797","24,506","25,295","24,540","24,699"
5,"19,111","15,426","27,117","24,782","25,404","24,232",NaN
6,"19,526","20,748","24,706","23,803","27,108","24,437",NaN
7,"18,759","29,776","25,381","24,638","27,228","23,978",NaN
8,"18,862","28,888","25,196","24,305","26,921","23,991",NaN
9,"19,302","26,450","24,557","25,084","26,152","23,913",NaN


## Find YOY percentage change of business applications in NYC for the years from 2019-2025


In [55]:
## create function for percent change. I used the one we did in class

def pct_chg(old_number, new_number):
    return round((new_number - old_number) / old_number * 100,1)
    


In [57]:
## create a for loop to calculate percentage change, and create an f'string for  new column called "yoy_pct_chg" of the years that are being compared
## Jesus helped me a LOT with this. I know it's something we learn next semester. I do understand it because we learned for loops in Javascript this semester.

years = [2019, 2020, 2021, 2022, 2023, 2024, 2025] 

for indx in range(len(years) -1):
    old_year =years[indx]
    next_year = years[indx + 1]
    col_name = f'yoy_pct_chg_{old_year}_{next_year}'
    pivoted_table[col_name] = pivoted_table.apply(lambda x: pct_chg(x[old_year], x[next_year]), axis = 1)

In [59]:
## call new table/dataframe
pivoted_table

Year,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,yoy_pct_chg_2019_2020,yoy_pct_chg_2020_2021,yoy_pct_chg_2021_2022,yoy_pct_chg_2022_2023,yoy_pct_chg_2023_2024,yoy_pct_chg_2024_2025
Month,,,,,,,,,,,,,,,,
1,"17,453","17,166","19,007","17,032","17,565","27,965","28,337","24,006","25,980","21,497",3,59,1,-15,8,-17
2,"16,828","17,295","18,825","19,465","19,522","24,374","25,371","25,438","24,594","23,729",0,25,4,0,-3,-4
3,"16,547","17,707","18,686","19,705","15,580","25,710","25,170","26,376","23,721","24,716",-21,65,-2,5,-10,4
4,"17,066","17,732","18,650","19,810","12,122","28,797","24,506","25,295","24,540","24,699",-39,138,-15,3,-3,1
5,"18,114","17,945","19,142","19,111","15,426","27,117","24,782","25,404","24,232",NaN,-19,76,-9,2,-5,NaN
6,"14,718","18,588","19,167","19,526","20,748","24,706","23,803","27,108","24,437",NaN,6,19,-4,14,-10,NaN
7,"18,070","18,251","19,374","18,759","29,776","25,381","24,638","27,228","23,978",NaN,59,-15,-3,10,-12,NaN
8,"16,651","18,336","18,805","18,862","28,888","25,196","24,305","26,921","23,991",NaN,53,-13,-4,11,-11,NaN
9,"17,711","17,778","18,965","19,302","26,450","24,557","25,084","26,152","23,913",NaN,37,-7,2,4,-9,NaN


In [ ]:
##Done!